In [41]:
import pandas as pd
import numpy as np
import json
import os
import sys
import pickle
from tqdm.notebook import tqdm as tqdm_n
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
import gc
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

In [18]:
data_path = 'data'

In [19]:
train_data = np.load(os.path.join(data_path, 'twitter_train_vectors.npy'), allow_pickle=True)
test_data = np.load(os.path.join(data_path, 'twitter_test_vectors.npy'), allow_pickle=True)
train_labels = np.load(os.path.join(data_path, 'twitter_train_labels.npy'), allow_pickle=True)
test_labels = np.load(os.path.join(data_path, 'twitter_test_labels.npy'), allow_pickle=True)

In [20]:
def average_on_window(data, label, size):
    new_data = []
    new_labels = []
    for i in tqdm_n(range(len(data))):
        sample = data[i]
        if len(sample) >= size:
            new_sample = []
            count = int(np.floor((len(sample) / size)))
            rest = int(len(sample) / size)
            for j in range(size-1):
                new_sample.append(np.array(sample[j*count:(j+1)*count].mean(axis=0)))
            if rest != 0:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count+rest].mean(axis=0)))
            else:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count].mean(axis=0)))
            new_data.append(np.array(new_sample))
            new_labels.append(label[i])
    return np.array(new_data), np.array(new_labels)

In [33]:
X_train, y_train = average_on_window(train_data, train_labels, 10)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [34]:
X_test, y_test = average_on_window(test_data, test_labels, 10)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [53]:
tf.random.set_seed(42)
np.random.seed(42)
model = Sequential()

model.add(LSTM(10, return_sequences = True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='selu'))
model.add(Dense(50, activation='selu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [54]:
callbacks = [EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [55]:
%%time
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=1024, callbacks=callbacks)

Train on 72990 samples, validate on 8111 samples
Epoch 1/50
72990/72990 [==============================] - 10s 137us/step - loss: 0.6149 - acc: 0.6560 - val_loss: 0.5720 - val_acc: 0.6998
Epoch 2/50
72990/72990 [==============================] - 9s 123us/step - loss: 0.5498 - acc: 0.7227 - val_loss: 0.5403 - val_acc: 0.7278
Epoch 3/50
72990/72990 [==============================] - 10s 134us/step - loss: 0.5304 - acc: 0.7367 - val_loss: 0.5292 - val_acc: 0.7374
Epoch 4/50
72990/72990 [==============================] - 10s 134us/step - loss: 0.5204 - acc: 0.7437 - val_loss: 0.5257 - val_acc: 0.7375
Epoch 5/50
72990/72990 [==============================] - 10s 140us/step - loss: 0.5142 - acc: 0.7480 - val_loss: 0.5199 - val_acc: 0.7421
Epoch 6/50
72990/72990 [==============================] - 10s 133us/step - loss: 0.5090 - acc: 0.7517 - val_loss: 0.5183 - val_acc: 0.7463
Epoch 7/50
72990/72990 [==============================] - 10s 135us/step - loss: 0.5045 - acc: 0.7533 - val_loss: 0.51

In [56]:
model.evaluate(X_test, y_test)

27074/27074 [==============================] - 3s 99us/step


[0.5207104603963931, 0.749722957611084]